# What does the key parameter do under the hood?

LaminDB is designed around associating biological metadata to artifacts and collections.
This enables querying for them in storage by metadata and removes the requirement for semantic artifact and collection names.

Here, we will discuss trade-offs for using the `key` parameter, which allows for semantic keys, in various scenarios.

## Setup

Install the `lamindb` Python package:
```shell
pip install 'lamindb[jupyter]'
```

We're simulating an artifact system with several nested folders and artifacts.
Such structures are resembled in, for example, the {doc}`docs:rxrx` guide.

In [ ]:
import random
import string
from pathlib import Path


def create_complex_biological_hierarchy(root_folder):
    root_path = Path(root_folder)

    if root_path.exists():
        print("Folder structure already exists. Skipping...")
    else:
        root_path.mkdir()

        raw_folder = root_path / "raw"
        preprocessed_folder = root_path / "preprocessed"
        raw_folder.mkdir()
        preprocessed_folder.mkdir()

        for i in range(1, 5):
            artifact_name = f"raw_data_{i}.txt"
            with (raw_folder / artifact_name).open("w") as f:
                random_text = "".join(
                    random.choice(string.ascii_letters) for _ in range(10)
                )
                f.write(random_text)

        for i in range(1, 3):
            collection_folder = raw_folder / f"Collection_{i}"
            collection_folder.mkdir()

            for j in range(1, 5):
                artifact_name = f"raw_data_{j}.txt"
                with (collection_folder / artifact_name).open("w") as f:
                    random_text = "".join(
                        random.choice(string.ascii_letters) for _ in range(10)
                    )
                    f.write(random_text)

        for i in range(1, 5):
            artifact_name = f"result_{i}.txt"
            with (preprocessed_folder / artifact_name).open("w") as f:
                random_text = "".join(
                    random.choice(string.ascii_letters) for _ in range(10)
                )
                f.write(random_text)


root_folder = "complex_biological_project"
create_complex_biological_hierarchy(root_folder)

In [ ]:
!lamin init --storage ./key-eval

In [ ]:
import lamindb as ln


ln.settings.verbosity = "hint"

In [ ]:
ln.UPath("complex_biological_project").view_tree()

In [ ]:
ln.settings.transform.stem_uid = "WIwaNDvlEkwS"
ln.settings.transform.version = "1"
ln.track()

## Storing artifacts using `Storage`, `File`, and `Collection`

Lamin has three storage classes that manage different types of in-memory and on-disk objects:

1. {class}`~lamindb.Storage`: Manages the default storage root that can be either local or in the cloud. For more details we refer to {doc}`docs:faq/storage`.
2. {class}`~lamindb.Artifact`: Manages datasets with an optional `key` that acts as a relative path within the current default storage root (see {class}`~lamindb.Storage`). An example is a single h5 artifact.
3. {class}`~lamindb.Collection`: Manages a collection of datasets with an optional `key` that acts as a relative path within the current default storage root (see {class}`~lamindb.Storage`). An example is a collection of h5 artifacts.

For more details we refer to {doc}`docs:tutorial`.

The current storage root is:

In [ ]:
ln.settings.storage

By default, Lamin uses virtual `keys` that are only reflected in the database but not in storage.
It is possible to turn this behavior off by setting `ln.settings.creation._artifact_use_virtual_keys = False`.
Generally, we discourage disabling this setting manually. For more details we refer to {doc}`docs:faq/storage`.

In [ ]:
ln.settings.creation._artifact_use_virtual_keys

We will now create `File` objects with and without semantic keys using `key` and also save them as `Collections`.

In [ ]:
artifact_no_key_1 = ln.Artifact("complex_biological_project/raw/raw_data_1.txt")
artifact_no_key_2 = ln.Artifact("complex_biological_project/raw/raw_data_2.txt")

The logging suggests that the artifacts will be saved to our current default storage with auto generated storage keys.

In [ ]:
artifact_no_key_1.save()
artifact_no_key_2.save()

In [ ]:
artifact_key_3 = ln.Artifact(
    "complex_biological_project/raw/raw_data_3.txt", key="raw/raw_data_3.txt"
)
artifact_key_4 = ln.Artifact(
    "complex_biological_project/raw/raw_data_4.txt", key="raw/raw_data_4.txt"
)
artifact_key_3.save()
artifact_key_4.save()

`Files` with keys are not stored in different locations because of the usage of `virtual keys`.
However, they are still semantically queryable by `key`.

In [ ]:
ln.Artifact.filter(key__contains="raw").df().head()

`Collection` does not have a `key` parameter because it does not store any additional data in `Storage`.
In contrast, it has a `name` parameter that serves as a semantic identifier of the collection.

In [ ]:
ds_1 = ln.Collection([artifact_no_key_1, artifact_no_key_2], name="no key collection")
ds_2 = ln.Collection([artifact_key_3, artifact_key_4], name="sample collection")
ds_1

## Advantages and disadvantages of semantic keys

Semantic keys have several advantages and disadvantages that we will discuss and demonstrate in the remaining notebook:

### Advantages:

- Simple: It can be easier to refer to specific collections in conversations
- Familiarity: Most people are familiar with the concept of semantic names

### Disadvantages

- Length: Semantic names can be long with limited aesthetic appeal
- Inconsistency: Lack of naming conventions can lead to confusion
- Limited metadata: Semantic keys can contain some, but usually not all metadata
- Inefficiency: Writing lengthy semantic names is a repetitive process and can be time-consuming
- Ambiguity: Overly descriptive artifact names may introduce ambiguity and redundancy
- Clashes: Several people may attempt to use the same semantic key. They are not unique

## Renaming artifacts

Renaming `Files` that have associated keys can be done on several levels.

### In storage

A artifact can be locally moved or renamed:

In [ ]:
artifact_key_3.path

In [ ]:
loaded_artifact = artifact_key_3.load()

In [ ]:
!mkdir complex_biological_project/moved_artifacts
!mv complex_biological_project/raw/raw_data_3.txt complex_biological_project/moved_artifacts

In [ ]:
artifact_key_3.path

After moving the artifact locally, the storage location (the path) has not changed and the artifact can still be loaded.

In [ ]:
artifact_3 = artifact_key_3.load()

The same applies to the `key` which has not changed.

In [ ]:
artifact_key_3.key

### By key

Besides moving the artifact in storage, the `key` can also be renamed.

In [ ]:
artifact_key_4.key

In [ ]:
artifact_key_4.key = "bad_samples/sample_data_4.txt"
artifact_key_4.key

Due to the usage of virtual `keys`, modifying the key does not change the storage location and the artifact stays accessible.

In [ ]:
artifact_key_4.path

In [ ]:
artifact_4 = artifact_key_4.load()

### Modifying the `path` attribute

However, modifying the `path` directly is not allowed:

In [ ]:
try:
    artifact_key_4.path = f"{ln.settings.storage}/here_now/sample_data_4.txt"
except AttributeError as e:
    print(e)

## Clashing semantic keys

Semantic keys should not clash. Let's attempt to use the same semantic key twice

In [ ]:
print(artifact_key_3.key)
print(artifact_key_4.key)

In [ ]:
artifact_key_4.key = "raw/raw_data_3.txt"

In [ ]:
print(artifact_key_3.key)
print(artifact_key_4.key)

When filtering for this semantic key it is now unclear to which artifact we were referring to:

In [ ]:
ln.Artifact.filter(key__icontains="sample_data_3").df()

When querying by `key` LaminDB cannot resolve which artifact we actually wanted.
In fact, we only get a single hit which does not paint a complete picture.

In [ ]:
print(artifact_key_3.uid)
print(artifact_key_4.uid)

Both artifacts still exist though with unique `uids` that can be used to get access to them.
Most importantly though, saving these artifacts to the database will result in an `IntegrityError` to prevent this issue.

In [ ]:
try:
    artifact_key_3.save()
    artifact_key_4.save()
except Exception as e:
    print(
        "It is not possible to save artifacts to the same key. This results in an"
        " Integrity Error!"
    )

We refer to {doc}`docs:faq/idempotency` for more detailed explanations of behavior when attempting to save artifacts multiple times.

## Hierarchies 

Another common use-case of `keys` are artifact hierarchies.
It can be useful to resemble the artifact structure in "complex_biological_project" from above also in LaminDB to allow for queries for artifacts that were stored in specific folders.
Common examples of this are folders specifying different processing stages such as `raw`, `preprocessed`, or `curated`.

Note that this use-case may also be overlapping with `Collection` which also allows for grouping `Files`.
However, `Collection` cannot model hierarchical groupings.

### Key

In [ ]:
import os

for root, _, artifacts in os.walk("complex_biological_project/raw"):
    for artifactname in artifacts:
        file_path = os.path.join(root, artifactname)
        key_path = file_path.removeprefix("complex_biological_project")
        ln_artifact = ln.Artifact(file_path, key=key_path)
        ln_artifact.save()

In [ ]:
ln.Artifact.filter(key__startswith="raw").df()

### Collection

Alternatively, it would have been possible to create a `Collection` with a corresponding name:

In [ ]:
all_data_paths = []
for root, _, artifacts in os.walk("complex_biological_project/raw"):
    for artifactname in artifacts:
        file_path = os.path.join(root, artifactname)
        all_data_paths.append(file_path)

all_data_artifacts = []
for path in all_data_paths:
    all_data_artifacts.append(ln.Artifact(path))

data_ds = ln.Collection(all_data_artifacts, name="data")
data_ds.save()

In [ ]:
ln.Collection.filter(name__icontains="data").df()

This approach will likely lead to clashes. Alternatively, `Ulabels` can be added to `Files` to resemble hierarchies.

### Ulabels

In [ ]:
for root, _, artifacts in os.walk("complex_biological_project/raw"):
    for artifactname in artifacts:
        file_path = os.path.join(root, artifactname)
        key_path = file_path.removeprefix("complex_biological_project")
        ln_artifact = ln.Artifact(file_path, key=key_path)
        ln_artifact.save()

        data_label = ln.ULabel(name="data")
        data_label.save()
        ln_artifact.ulabels.add(data_label)

In [ ]:
labels = ln.ULabel.lookup()

In [ ]:
ln.Artifact.filter(ulabels__in=[labels.data]).df()

However, `Ulabels` are too versatile for such an approach and clashes are also to be expected here.

### Metadata

Due to the chance of clashes for the aforementioned approaches being rather high, we generally recommend not to store hierarchical data with solely semantic keys.
Biological metadata makes `Files` and `Collections` unambiguous and easily queryable.


## Legacy data and multiple storage roots

### Distributed Collections

LaminDB can ingest legacy data that already had a structure in their storage.
In such cases, it disables `_artifact_use_virtual_keys` and the artifacts are ingested with their actual storage location.
It might be therefore be possible that `Files` stored in different storage roots may be associated with a single `Collection`.
To simulate this, we are disabling `_artifact_use_virtual_keys` and ingest artifacts stored in a different path (the "legacy data").

In [ ]:
ln.settings.creation._artifact_use_virtual_keys = False

In [ ]:
for root, _, artifacts in os.walk("complex_biological_project/preprocessed"):
    for artifactname in artifacts:
        file_path = os.path.join(root, artifactname)
        key_path = file_path.removeprefix("complex_biological_project")

        print(file_path)
        print()

        ln_artifact = ln.Artifact(file_path, key=f"./{key_path}")
        ln_artifact.save()

In [ ]:
ln.Artifact.df()

In [ ]:
artifact_from_raw = ln.Artifact.filter(key__icontains="Collection_2/raw_data_1").first()
artifact_from_preprocessed = ln.Artifact.filter(
    key__icontains="preprocessed/result_1"
).first()

print(artifact_from_raw.path)
print(artifact_from_preprocessed.path)

Let's create our `Collection`:

In [ ]:
ds = ln.Collection(
    [artifact_from_raw, artifact_from_preprocessed], name="raw_and_processed_collection_2"
)
ds.save()

In [ ]:
ds.artifacts.df()

### Modeling directories

In [ ]:
ln.settings.creation._artifact_use_virtual_keys = True

In [ ]:
dir_path = ln.core.datasets.dir_scrnaseq_cellranger("sample_001")
ln.UPath(dir_path).view_tree()

There are two ways to create `Artifact` objects from directories: {func}`~lamindb.Artifact.from_dir` and {class}`~lamindb.Artifact`.

In [ ]:
cellranger_raw_artifact = ln.Artifact.from_dir("sample_001/raw_feature_bc_matrix/")

In [ ]:
for artifact in cellranger_raw_artifact:
    artifact.save()

In [ ]:
cellranger_raw_folder = ln.Artifact(
    "sample_001/raw_feature_bc_matrix/", description="cellranger raw"
)
cellranger_raw_folder.save()

In [ ]:
ln.Artifact.filter(key__icontains="raw_feature_bc_matrix").df()

In [ ]:
ln.Artifact.filter(key__icontains="raw_feature_bc_matrix/matrix.mtx.gz").one().path

In [ ]:
artifact = ln.Artifact.filter(description="cellranger raw").one()
artifact.path.glob("*")